<a href="https://colab.research.google.com/github/DigvjSingh/DigvjSingh/blob/main/JUL3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# ------------------------ Parameters ------------------------
num_elements = 11
d = 0.5
wavelength = 1
noise_power = 0.01
num_snapshots = 100
num_samples = 1000

# ------------------------ Signal Generator ------------------------
def generate_received_signal(angles_deg, num_elements, d, wavelength, noise_power, snapshots):
    angles_rad = np.radians(angles_deg)
    A = np.exp(1j * 2 * np.pi * d * np.outer(np.arange(num_elements), np.sin(angles_rad)) / wavelength)
    S = np.random.randn(len(angles_deg), snapshots) + 1j * np.random.randn(len(angles_deg), snapshots)
    X = A @ S
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*X.shape) + 1j * np.random.randn(*X.shape))
    noisy_signal = X + noise
    return np.stack([np.real(noisy_signal), np.imag(noisy_signal)], axis=-1)  # (elements, snapshots, 2)

# ------------------------ Data Creation ------------------------
X, Y = [], []
for _ in range(num_samples):
    angle_pair = np.sort(np.random.uniform(-90, 90, 2))  # two DoAs
    signal = generate_received_signal(angle_pair, num_elements, d, wavelength, noise_power, num_snapshots)
    X.append(signal)
    Y.append(angle_pair)

X = np.array(X)                                # (samples, elements, snapshots, 2)
X = np.transpose(X, (0, 2, 1, 3))              # (samples, snapshots, elements, 2)
X = X[:, :, np.newaxis, :, :]                  # Add height dim: (samples, snapshots, 1, elements, 2)
Y = np.radians(np.array(Y)) / np.pi            # Normalize to [-1, 1]

# ------------------------ Split ------------------------
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# ------------------------ ConvLSTM Model ------------------------
def build_model(input_shape):
    model = keras.Sequential([
        layers.ConvLSTM2D(filters=32, kernel_size=(1, 3), activation='relu',
                          input_shape=input_shape, return_sequences=False),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(2)  # Predict two angles
    ])
    return model

model = build_model(X_train.shape[1:])

# ------------------------ Hybrid Loss ------------------------
def hybrid_loss(y_true, y_pred):
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))
    mse = tf.reduce_mean(tf.square(y_true - y_pred))
    return 0.5 * mae + 0.5 * mse

model.compile(optimizer=keras.optimizers.Adam(0.001), loss=hybrid_loss)

# ------------------------ Train ------------------------
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_val, Y_val))

# ------------------------ Evaluate ------------------------
loss = model.evaluate(X_test, Y_test)
print("📉 Final Test Loss (hybrid MAE+MSE, radians/π):", loss)

# ------------------------ Predict ------------------------
preds = model.predict(X_test)
true_deg = np.degrees(Y_test * np.pi)
pred_deg = np.degrees(preds * np.pi)
mae_per_angle = np.mean(np.abs(true_deg - pred_deg), axis=0)
mae_total = mae_per_angle.mean()

print(f"\n✅ MAE per angle (degrees): {mae_per_angle}")
print(f"🎯 Total Mean Absolute Error (degrees): {mae_total}")

# ------------------------ Sample Prediction ------------------------
idx = np.random.randint(0, len(X_test))
print(f"\n🔍 True Angles (deg): {true_deg[idx]}")
print(f"📈 Predicted Angles (deg): {pred_deg[idx]}")
print("🧮 MAE on this sample (deg):", np.mean(np.abs(true_deg[idx] - pred_deg[idx])))

# ------------------------ Angle-wise Error Plot ----------------


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 251ms/step - loss: 0.8966 - val_loss: 0.1958
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 189ms/step - loss: 0.5545 - val_loss: 0.1790
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 209ms/step - loss: 0.3972 - val_loss: 0.1756
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 276ms/step - loss: 0.2753 - val_loss: 0.1747
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 212ms/step - loss: 0.2448 - val_loss: 0.1735
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 184ms/step - loss: 0.1953 - val_loss: 0.1722
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - loss: 0.1745 - val_loss: 0.1699
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 183ms/step - loss: 0.1656 - val_loss: 0.1691
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - loss: 0.1473 - val_loss: 0.1671
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - loss: 0.1315 - val_loss: 0.1667
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 207ms/step - loss: 0.1255 - val_loss: 0.1644
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 273ms/ste

NameError: name 'plot' is not defined